In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
from plotly.subplots import make_subplots


from sqlalchemy import create_engine

In [2]:
HOST = '37.139.42.145'
DBNAME = 'game-analytics'
USER = 'analytics'
PASSWORD = 'BRtTaqYiJyr29WXN'
TABLE_SCHEMA = 'diploma'
ENGINE = create_engine(f'postgresql://{USER}:{PASSWORD}@{HOST}/{DBNAME}')

query_users = f'''SELECT * FROM {TABLE_SCHEMA}.users'''
query_geo_daily_report = f'''SELECT * FROM {TABLE_SCHEMA}.geo_daily_report'''

In [5]:
## Считаем lifetime

***Lifetime это сумма кол-во дней retention  в среднем кот играл 1 user ***

In [3]:
# Считаем Lifetime

data_start ='2022-04-01'
data_end ='2022-05-01'
data_project_id = 'MiT'

start_time = datetime.now()

Lifetime_user_type_sql = pd.read_sql(f''' 
WITH
-- разделяем когорты по типу трафика и по условию
    user_types AS (
        SELECT
            *,
            CASE WHEN is_organic = TRUE THEN appsflyer_id END AS organic,
            CASE WHEN is_organic != TRUE THEN appsflyer_id END AS non_organic 
        FROM {TABLE_SCHEMA}.users 
        WHERE
            1=1
            AND project_id = '{data_project_id}' 
            AND install_date >='{data_start}'
            AND install_date < '{data_end}'
),
last_date AS (
-- даты всех логинов игроков
    SELECT
        DISTINCT
            appsflyer_id,
            (install_date)::date AS ld,
            organic,
            non_organic            
    FROM
        user_types
),
first_date AS (
-- дата первого входа
    SELECT
        appsflyer_id,
        organic,
        non_organic, 
        install_date::date AS fd
    FROM
        user_types
),
rate AS (
-- джоин все на все и вычитаем интервалы ld и fd
    SELECT
      -- last_date.appsflyer_id,
       last_date.organic,
       last_date.non_organic,
       ld - fd AS retention_day
    FROM
        first_date
    JOIN
        last_date ON last_date.appsflyer_id = first_date.appsflyer_id   		
),
ret_rate AS (
SELECT
-- делим на кол-во игроков 0 дня для organic - 13964 и для non_organic - 6627

    retention_day,
    ROUND((COUNT(DISTINCT organic)/13964::FLOAT)::NUMERIC,2) AS ret_rate_organic,
	ROUND((COUNT(DISTINCT non_organic)/6627::FLOAT)::NUMERIC,2) as ret_rate_non_organic    
FROM 
    rate
WHERE
    1=1
    AND retention_day >= 0
GROUP BY 1
)

-- получаем lifetime сумму кол-во дней в среднем, кот. играл один user 2.285 и 2.034
SELECT
    SUM(ret_rate_organic) as lifetime_organic,
    SUM(ret_rate_non_organic) lifetime_non_organic
FROM
ret_rate
''', con=ENGINE)

print('execution time: %s' % str(datetime.now() - start_time))
Lifetime_user_type_sql.head(3)

execution time: 0:00:01.678637


,lifetime_organic,lifetime_non_organic
0,1.02,1.01
